In [3]:
from scipy.optimize import fsolve

def find_velocity(total_power, air_density, cda_value, CRR, total_mass, gravity, length_segment_m, friction_loss, gravitational_power):
    def equations(velocity):
        air_resistance = 0.5 * air_density * cda_value * (velocity ** 3)
        rolling_resistance = CRR * total_mass * gravity * velocity
        time_sec = length_segment_m / velocity
        gravitational_power_watt = gravitational_power / time_sec
        total_power_watt = (1 + friction_loss) * (air_resistance + rolling_resistance + gravitational_power_watt)
        return total_power_watt - total_power

    velocity_initial_guess = 5  # Initial guess for velocity
    velocity_solution = fsolve(equations, velocity_initial_guess)
    return velocity_solution[0]

def calculate_climbing_duration(time_sec=None, relative_power=None, length_segment_km=None, elevation_gain_m=None, rider=None, drafting=None):
    # Constants
    air_density = 1.15
    CRR = 0.004
    gravity = 9.81  # m/s^2
    additional_mass = 7.8  # Additional mass for bike/equipment
    friction_loss = 0.02

    if length_segment_km is None or elevation_gain_m is None or rider is None or drafting is None:
        return "Please specify the length of the segment (in km), elevation gain, rider details, and drafting condition."

    # Convert length from kilometers to meters
    length_segment_m = length_segment_km * 1000

    # Extract rider details
    weight_rider = rider.get("weight_rider")
    cda_values = rider.get("cda_values", {})
    cda_value = cda_values.get(drafting, cda_values.get("Full Draft"))

    if weight_rider is None or cda_value is None:
        return "Please specify the weight of the rider and CdA values for the drafting condition."

    total_mass = weight_rider + additional_mass  # Total mass including rider and equipment

    if time_sec is not None:
        # Calculate velocity
        velocity = length_segment_m / time_sec

        # Air Resistance Calculation
        air_resistance_watt = 0.5 * air_density * cda_value * (velocity ** 3)

        # Rolling Resistance Calculation
        rolling_resistance_watt = CRR * total_mass * gravity * velocity

        # Gravitational Power Calculation
        gravitational_power_watt = (total_mass * gravity * elevation_gain_m) / time_sec

        # Total Power Calculation
        total_power_watt = (1 + friction_loss) * (air_resistance_watt + rolling_resistance_watt + gravitational_power_watt)

        # Relative Power Calculation
        relative_power_watt_per_kg = total_power_watt / weight_rider

#         print("Relative Power (W/kg): " + str(relative_power_watt_per_kg))
        return relative_power_watt_per_kg

    elif relative_power is not None:
        # Calculate total power from relative power per kg
        total_power_watt = relative_power * weight_rider

        # Gravitational power portion
        gravitational_power = (total_mass * gravity * elevation_gain_m)

        velocity = find_velocity(total_power_watt, air_density, cda_value, CRR, total_mass, gravity, length_segment_m, friction_loss, gravitational_power)

        # Calculate time required
        time_required = round(length_segment_m / velocity)

#         print("Time Required (sec): " + str(time_required))
        return time_required

    else:
        return "Please specify either time in seconds or relative power, not both."

# # Example usage
# time_input = None  # seconds
# relative_power_input = 5.7615  # Specify one of the two
# length_segment_input = 9.3  # in km
# elevation_gain_input = 680  # in meters
# drafting = "Full Draft"

rider = {
    "weight_rider": 59,  # in kg
    "cda_values": {
        "Full Draft": 0.2625,
        "Semi Draft": 0.305,
        "No Draft": 0.35
    }
}

# output = calculate_climbing_duration(time_sec=time_input, relative_power=relative_power_input, length_segment_km=length_segment_input, elevation_gain_m=elevation_gain_input, rider=rider, drafting=drafting)


1560

In [104]:
def define_drafting_decisions(segments, semi_draft_point=0.6, full_draft_point=0.9):
    num_segments = len(segments)

    if isinstance(semi_draft_point, float):
        semi_draft_segment = int(num_segments * semi_draft_point)
    elif isinstance(semi_draft_point, int):
        semi_draft_segment = semi_draft_point
    else:
        raise ValueError("The semi draft point must either be an integer or a float")

    if isinstance(full_draft_point, float):
        full_draft_segment = int(num_segments * full_draft_point)
    elif isinstance(full_draft_point, int):
        full_draft_segment = full_draft_point
    else:
        raise ValueError("The full draft point must either be an integer or a float")

    segments['Drafting'] = ''

    for i in range(num_segments):
        if i < semi_draft_segment:
            segments.loc[i, 'Drafting'] = 'Full Draft'
        elif i < full_draft_segment:
            segments.loc[i, 'Drafting'] = 'Semi Draft'
        else:
            segments.loc[i, 'Drafting'] = 'No Draft'

    return segments, semi_draft_segment, full_draft_segment

In [98]:
import pandas as pd
segments = pd.read_excel("../Segment Automation/TdF-stages/excel_output/stage-1-segments.xlsx")
segments

,Segment,Start Altitude (m),End Altitude (m),Start Point (km),End Point (km),Total Distance (km),Average Slope (%),Drafting
0,Segment_1,63,151,0.000000,27.425682,27.425682,0.320867,No Draft
1,Segment_2,151,341,27.425682,37.309794,9.884112,1.922277,No Draft
2,Segment_3,341,733,37.309794,43.445983,6.136189,6.388330,No Draft
3,Segment_4,733,981,43.445983,48.690183,5.244200,4.729034,No Draft
4,Segment_5,981,790,48.690183,53.060352,4.370170,-4.370540,No Draft
5,Segment_6,790,772,53.060352,54.078858,1.018506,-1.767295,No Draft
6,Segment_7,772,553,54.078858,59.210064,5.131206,-4.268003,No Draft
7,Segment_8,553,424,59.210064,64.849972,5.639908,-2.287271,No Draft
8,Segment_9,424,429,64.849972,65.910426,1.060454,0.471496,No Draft
9,Segment_10,429,302,65.910426,72.214065,6.303639,-2.014709,No Draft


In [107]:
segments,sd,fd = define_drafting_decisions(segments.drop(columns = "Drafting"), semi_draft_point=0.6, full_draft_point=0.9)
segments["Relative Power"] = 5.5

In [112]:
def apply_descending_duration(row, average_speed):
    return row["Total Distance (km)"]*3600/average_speed


def apply_climbing_duration(row):
    # Define the logic for the negative elevation gain
    return calculate_climbing_duration(
        time_sec=None,
        relative_power=row["Relative Power"],
        length_segment_km=row["Total Distance (km)"],
        elevation_gain_m=abs(row["End Altitude (m)"] - row["Start Altitude (m)"]),
        rider=rider,
        drafting=row["Drafting"]
    )

def apply_duration(row, average_speed=50):
    if row["End Altitude (m)"] >= row["Start Altitude (m)"]:
        return apply_climbing_duration(row)
    else:
        return apply_descending_duration(row,average_speed)

segments['Duration'] = segments.apply(apply_duration, axis=1)

In [120]:
def compute_glycogen_level(segments, glycogen_start_level=100):
    new_column = []
    previous_value = glycogen_start_level

    for index, row in segments.iterrows():
        if index == 0:
            new_value = glycogen_start_level
        elif row["Average Slope (%)"] < 0:
            new_value = previous_value
        else:
            new_value = previous_value*row["Relative Power"]/6   # Example function: summing previous value and parameter
        new_column.append(new_value)
        previous_value = new_value

    segments['Glycogen Level (%)'] = new_column
    return segments

In [121]:
compute_glycogen_level(segments)

,Segment,Start Altitude (m),End Altitude (m),Start Point (km),End Point (km),Total Distance (km),Average Slope (%),Drafting,Relative Power,Duration,Glycogen Level (%)
0,Segment_1,63,151,0.000000,27.425682,27.425682,0.320867,Full Draft,5.5,2284.000000,100.000000
1,Segment_2,151,341,27.425682,37.309794,9.884112,1.922277,Full Draft,5.5,965.000000,91.666667
2,Segment_3,341,733,37.309794,43.445983,6.136189,6.388330,Full Draft,5.5,974.000000,84.027778
3,Segment_4,733,981,43.445983,48.690183,5.244200,4.729034,Full Draft,5.5,696.000000,77.025463
4,Segment_5,981,790,48.690183,53.060352,4.370170,-4.370540,Full Draft,5.5,314.652225,77.025463
5,Segment_6,790,772,53.060352,54.078858,1.018506,-1.767295,Full Draft,5.5,73.332425,77.025463
6,Segment_7,772,553,54.078858,59.210064,5.131206,-4.268003,Full Draft,5.5,369.446819,77.025463
7,Segment_8,553,424,59.210064,64.849972,5.639908,-2.287271,Full Draft,5.5,406.073362,77.025463
8,Segment_9,424,429,64.849972,65.910426,1.060454,0.471496,Full Draft,5.5,90.000000,70.606674
9,Segment_10,429,302,65.910426,72.214065,6.303639,-2.014709,Full Draft,5.5,453.862010,70.606674
